In [2]:
import pulp

def compute_coverage(positions, directions, camera_types, grid_size, obstacles):
    coverage = {}
    row_labels = 'ABC'
    col_labels = range(1, 6)
    
    # 方向ベクトル（仮の方向設定：上下左右と斜め）
    direction_vectors = {
        "a_1": (0, 1), "a_2": (1, 0), "a_3": (0, -1), "a_4": (-1, 0),
        "b_1": (1, 1), "b_2": (-1, -1)
    }
    
    for pos in positions:
        for cam_type, cam_dirs in directions.items():
            for dir in cam_dirs:
                dx, dy = direction_vectors[dir]
                covered_cells = []
                # 仮定として、各方向に3セルまでカバーできるとする
                for step in range(1, 4):
                    new_row = pos[0] + dx * step
                    new_col = pos[1] + dy * step
                    if 0 <= new_row < grid_size[0] and 0 <= new_col < grid_size[1]:
                        cell_name = f"{row_labels[new_row]}{new_col + 1}"
                        if cell_name not in obstacles:
                            covered_cells.append(cell_name)
                        else:
                            break
                coverage[(pos, cam_type, dir)] = covered_cells
    return coverage

# 初期設定
camera_types = {"A": 100, "B": 150}
directions = {"A": ["a_1", "a_2", "a_3", "a_4"], "B": ["b_1", "b_2"]}
positions = [(0, 0), (1, 1), (2, 2)]  # グリッド座標
obstacles = {"B2", "C3"}  # 障害物
grid_size = (3, 5)  # グリッドの大きさ

# カメラのカバレッジを計算
coverage = compute_coverage(positions, directions, camera_types, grid_size, obstacles)

# PuLP問題の設定
prob = pulp.LpProblem("CameraPlacement", pulp.LpMaximize)
x = pulp.LpVariable.dicts("x", ((i, j, d) for i in positions for j in camera_types for d in directions[j]), cat="Binary")
print("x", x)
z = pulp.LpVariable.dicts("z", (f"{r}{c+1}" for r in 'ABC' for c in range(5)), cat="Binary")

# 目的関数
prob += pulp.lpSum([z[a] for a in z])

# 予算制約
prob += pulp.lpSum([camera_types[j] * x[(i, j, d)] for i in positions for j in camera_types for d in directions[j]]) <= 500

# 配置制約
for i in positions:
    for j in camera_types:
        prob += pulp.lpSum([x[(i, j, d)] for d in directions[j]]) <= 1

# カバレッジ制約
for a in z:
    prob += z[a] <= pulp.lpSum([x[(i, j, d)] for (i, j, d) in coverage if a in coverage[(i, j, d)]])

# 問題を解く
prob.solve()

# 結果の出力
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

# 監視可能面積の最大化の結果
print("Maximized Covered Area:", pulp.value(prob.objective))


x {((0, 0), 'A', 'a_1'): x_((0,_0),_'A',_'a_1'), ((0, 0), 'A', 'a_2'): x_((0,_0),_'A',_'a_2'), ((0, 0), 'A', 'a_3'): x_((0,_0),_'A',_'a_3'), ((0, 0), 'A', 'a_4'): x_((0,_0),_'A',_'a_4'), ((0, 0), 'B', 'b_1'): x_((0,_0),_'B',_'b_1'), ((0, 0), 'B', 'b_2'): x_((0,_0),_'B',_'b_2'), ((1, 1), 'A', 'a_1'): x_((1,_1),_'A',_'a_1'), ((1, 1), 'A', 'a_2'): x_((1,_1),_'A',_'a_2'), ((1, 1), 'A', 'a_3'): x_((1,_1),_'A',_'a_3'), ((1, 1), 'A', 'a_4'): x_((1,_1),_'A',_'a_4'), ((1, 1), 'B', 'b_1'): x_((1,_1),_'B',_'b_1'), ((1, 1), 'B', 'b_2'): x_((1,_1),_'B',_'b_2'), ((2, 2), 'A', 'a_1'): x_((2,_2),_'A',_'a_1'), ((2, 2), 'A', 'a_2'): x_((2,_2),_'A',_'a_2'), ((2, 2), 'A', 'a_3'): x_((2,_2),_'A',_'a_3'), ((2, 2), 'A', 'a_4'): x_((2,_2),_'A',_'a_4'), ((2, 2), 'B', 'b_1'): x_((2,_2),_'B',_'b_1'), ((2, 2), 'B', 'b_2'): x_((2,_2),_'B',_'b_2')}
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ikumadaiki/python/causal_learning/.venv/lib/python3.9/site-packages/pulp

In [15]:
import pulp

# 問題のインスタンスを作成
prob = pulp.LpProblem("CameraPlacement", pulp.LpMaximize)

# カメラタイプとコスト
camera_types = {"A": 100, "B": 150}
# 位置と向きの例（仮定）
positions = [1, 2, 3]  # 3行目までの位置
directions = {"A": ["a_1", "a_2", "a_3", "a_4"], "B": ["b_1", "b_2"]}  # タイプ別の向き

# セルの集合（3x5グリッド）
cells = ["A1", "A2", "A3", "A4", "A5", "B1", "B2", "B3", "B4", "B5", "C1", "C2", "C3", "C4", "C5"]

# 各カメラがカバーできるセルのセット（仮定、障害物を考慮して調整）
coverage = {
    (1, "A", "a_1"): ["A1", "A2", "B1", "B3"],
    (1, "A", "a_2"): ["A3", "A4", "A5"],
    (1, "B", "b_1"): ["B1", "B3", "B4", "B5"],
    (1, "B", "b_2"): ["A1", "A2", "A3", "C1", "C3"],
    (2, "A", "a_3"): ["B1", "C1"],
    (2, "A", "a_4"): ["B3", "B4", "C3", "C4"],
    (2, "B", "b_1"): ["A4", "A5", "B4", "B5"],
    (2, "B", "b_2"): ["C1", "C2", "C3", "C4", "C5"],
    (3, "A", "a_1"): ["C4", "C5"],
    (3, "A", "a_2"): ["C1", "C2", "C3"],
    (3, "B", "b_1"): ["C3", "C4", "C5"],
    (3, "B", "b_2"): ["C3", "C4", "C5"]
}

# 利用可能な予算
budget = 500

# 変数の定義
x = pulp.LpVariable.dicts("x", ((i, j, d) for i in positions for j in camera_types for d in directions[j]), 
                          cat="Binary")
z = pulp.LpVariable.dicts("z", (a for a in cells), cat="Binary")

# 目的関数
prob += pulp.lpSum([z[a] for a in cells])

# 予算制約
prob += pulp.lpSum([camera_types[j] * x[(i, j, d)] for i in positions for j in camera_types for d in directions[j]]) <= budget

# 配置制約
for i in positions:
    for j in camera_types:
        prob += pulp.lpSum([x[(i, j, d)] for d in directions[j]]) <= 1

# カバレッジ制約（障害物を考慮しているため特定のセルは除外されています）
for a in cells:
    prob += z[a] <= pulp.lpSum([x[(i, j, d)] for i, j, d in coverage if a in coverage[(i, j, d)]])

# 問題を解く
prob.solve()

# 結果の出力
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

# 監視可能面積の最大化の結果
print("Maximized Covered Area:", pulp.value(prob.objective))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ikumadaiki/python/causal_learning/.venv/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j4/j781gq2d1wd49m_sglw4f8580000gn/T/6b9be6bfa86044348f6fbe04784cad12-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/j4/j781gq2d1wd49m_sglw4f8580000gn/T/6b9be6bfa86044348f6fbe04784cad12-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 27 COLUMNS
At line 202 RHS
At line 225 BOUNDS
At line 259 ENDATA
Problem MODEL has 22 rows, 33 columns and 93 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 14 - 0.00 seconds
Cgl0002I 1 variables fixed
Cgl0004I processed model has 20 rows, 25 columns (25 integer (25 of which binary)) and 74 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 8 integers unsatisfied sum - 4
Cbc0038

In [2]:
# 結果の出力
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

# 監視可能面積の最大化の結果
print("Maximized Covered Area:", pulp.value(prob.objective))

Status: Optimal
x_((0,_0),_'A',_'a_1') = 1.0
x_((1,_1),_'A',_'a_1') = 1.0
x_((1,_1),_'B',_'b_2') = 1.0
x_((2,_2),_'A',_'a_1') = 1.0
z_A1 = 1.0
z_A2 = 1.0
z_A3 = 1.0
z_A4 = 1.0
z_B3 = 1.0
z_B4 = 1.0
z_B5 = 1.0
z_C4 = 1.0
z_C5 = 1.0
Maximized Covered Area: 9.0
